# 그룹화

In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({'ID': [1, 2, 3, 4, 5], '가입일': ['2021-01-02', '2021-01-04', '2021-01-10', '2021-02-10', '2021-02-24'], '성별': ['F', 'M', 'F', 'M', 'M']})
df2 = pd.DataFrame({'구매순서': [1, 2, 3, 4, 5], 'ID': [1, 1, 2, 4, 1], '구매월': [1, 1, 2, 2, 3], '금액': [1000, 1500, 2000, 3000, 4000]})

df1: 회원 정보 저장 프레임  
df2: 구매 내역 저장 프레임

각 회원의 정보와 구매 내역을 취합하여 하나의 데이터프레임으로 생성하기

In [3]:
pd.merge(df1, df2, how='left', on='ID')

,ID,가입일,성별,구매순서,구매월,금액
0,1,2021-01-02,F,1.0,1.0,1000.0
1,1,2021-01-02,F,2.0,1.0,1500.0
2,1,2021-01-02,F,5.0,3.0,4000.0
3,2,2021-01-04,M,3.0,2.0,2000.0
4,3,2021-01-10,F,NaN,NaN,NaN
5,4,2021-02-10,M,4.0,2.0,3000.0
6,5,2021-02-24,M,NaN,NaN,NaN


df1과 df2를 합하여 각 회원의 누적 금액을 회원 ID 별로 구하기

In [4]:
df2

,구매순서,ID,구매월,금액
0,1,1,1,1000
1,2,1,1,1500
2,3,2,2,2000
3,4,4,2,3000
4,5,1,3,4000


In [5]:
df2.groupby(by=['ID'])['금액'].sum()

ID
1    6500
2    2000
4    3000
Name: 금액, dtype: int64

In [10]:
type(df2.groupby(by=['ID'])['금액'].sum())

pandas.core.series.Series

In [6]:
s2 = df2.groupby(by=['ID'])['금액'].sum()

In [7]:
pd.merge(df1, s2, how='left', on='ID')

,ID,가입일,성별,금액
0,1,2021-01-02,F,6500.0
1,2,2021-01-04,M,2000.0
2,3,2021-01-10,F,NaN
3,4,2021-02-10,M,3000.0
4,5,2021-02-24,M,NaN


각 회원의 월별 누적 금액을 회원 ID 별로 구하기

In [8]:
df2.groupby(by=['ID', '구매월'])['금액'].sum()

ID  구매월
1   1      2500
    3      4000
2   2      2000
4   2      3000
Name: 금액, dtype: int64

In [9]:
type(df2.groupby(by=['ID', '구매월'])['금액'].sum())

pandas.core.series.Series

In [11]:
s2 = df2.groupby(by=['ID', '구매월'])['금액'].sum()

In [13]:
s2.index    # id와 구매월에 대한 멀티인덱스

MultiIndex([(1, 1),
            (1, 3),
            (2, 2),
            (4, 2)],
           names=['ID', '구매월'])

In [15]:
pd.merge(df1, s2, how='left', on='ID')    # 구매월 데이터가 빠져있음

,ID,가입일,성별,금액
0,1,2021-01-02,F,2500.0
1,1,2021-01-02,F,4000.0
2,2,2021-01-04,M,2000.0
3,3,2021-01-10,F,NaN
4,4,2021-02-10,M,3000.0
5,5,2021-02-24,M,NaN


구매월도 포함되도록 변경

In [17]:
df3 = pd.DataFrame(s2)
df3     # 여전이 Id, 구매월이 인덱스

금액
ID 구매월      
1  1    2500
   3    4000
2  2    2000
4  2    3000

In [19]:
pd.merge(df1, df3, how='left', on='ID')   # 여전히 구매월 없음

,ID,가입일,성별,금액
0,1,2021-01-02,F,2500.0
1,1,2021-01-02,F,4000.0
2,2,2021-01-04,M,2000.0
3,3,2021-01-10,F,NaN
4,4,2021-02-10,M,3000.0
5,5,2021-02-24,M,NaN


그룹을 index로 사용하고 싶지 않은 경우에는 `as_index = False`로 설정

In [21]:
df2.groupby(by=['ID', '구매월'], as_index=False)['금액'].sum()

,ID,구매월,금액
0,1,1,2500
1,1,3,4000
2,2,2,2000
3,4,2,3000


In [22]:
type(df2.groupby(by=['ID', '구매월'], as_index=False)['금액'].sum())

pandas.core.frame.DataFrame

In [23]:
df3 = df2.groupby(by=['ID', '구매월'], as_index=False)['금액'].sum()

In [25]:
pd.merge(df1, df3, how='left', on='ID')   # 구매월까지 포함된 결과 출력

,ID,가입일,성별,구매월,금액
0,1,2021-01-02,F,1.0,2500.0
1,1,2021-01-02,F,3.0,4000.0
2,2,2021-01-04,M,2.0,2000.0
3,3,2021-01-10,F,NaN,NaN
4,4,2021-02-10,M,2.0,3000.0
5,5,2021-02-24,M,NaN,NaN


각 회원의 구매 내역을 저장한 데이터프레임 df를 통해 각 회원의 누적 금액과 누적 구매 회수를 회원 ID 별로 구하기  

In [26]:
df = pd.DataFrame({'구매순서': [1, 2, 3, 4, 5], 'ID': [1, 1, 2, 4, 1], '구매월': [1, 1, 2, 2, 3], '금액': [1000, 1500, 2000, 3000, 4000], '수수료': [100, 150, 200, 300, 400]})

In [27]:
df

,구매순서,ID,구매월,금액,수수료
0,1,1,1,1000,100
1,2,1,1,1500,150
2,3,2,2,2000,200
3,4,4,2,3000,300
4,5,1,3,4000,400


In [33]:
df2 = df.groupby(by=['ID'])['금액'].agg([sum, len])
df2

,sum,len
ID,,
1,6500,3
2,2000,1
4,3000,1


In [34]:
df2.reset_index(inplace=True)
df2

,ID,sum,len
0,1,6500,3
1,2,2000,1
2,4,3000,1


df가 각 회원의 구매 내역을 저장한 데이터 프레임일 때 각 회원의 최대 사용 금액 / 최소 사용 금액과 최저 수수료의 값을 구하기

In [35]:
df

,구매순서,ID,구매월,금액,수수료
0,1,1,1,1000,100
1,2,1,1,1500,150
2,3,2,2,2000,200
3,4,4,2,3000,300
4,5,1,3,4000,400


In [42]:
df2 = df.groupby(by=['ID']).agg({'금액': [max, min], '수수료': [min]})
df2

금액        수수료
     max   min  min
ID                 
1   4000  1000  100
2   2000  2000  200
4   3000  3000  300

In [45]:
df2.reset_index()   # ID는 column이 되었는데 금액의 (max, min)과 수수료 (min)이 잘 처리되지 않음

ID    금액        수수료
       max   min  min
0  1  4000  1000  100
1  2  2000  2000  200
2  4  3000  3000  300

In [46]:
df2.columns   # column이 multi_index

MultiIndex([( '금액', 'max'),
            ( '금액', 'min'),
            ('수수료', 'min')],
           )

In [47]:
df2.columns.values

array([('금액', 'max'), ('금액', 'min'), ('수수료', 'min')], dtype=object)

In [48]:
df2.columns = ['_'.join(col) for col in df2.columns.values]

In [49]:
df2

,금액_max,금액_min,수수료_min
ID,,,
1,4000,1000,100
2,2000,2000,200
4,3000,3000,300


In [50]:
df2.reset_index(inplace=True)
df2

,ID,금액_max,금액_min,수수료_min
0,1,4000,1000,100
1,2,2000,2000,200
2,4,3000,3000,300
